In [1]:
import pandas as pd

In [46]:
df = pd.read_csv("Cleaned_V1.csv")

In [2]:
df.shape

### Group sku to product

In [4]:
## delete the record with FALSE variant_sku
df = df.drop(54)

In [1]:
df["grouped_product"] = df["variant_sku"].apply(lambda x: int(x[0:2]) if type(x)==str else "")
def create_name(group):
    if group == 21:
        name = "Women's +20"
    elif group == 41:
        name = "Women's +40"
    elif group == 40:
        name = "Men's +40"
    elif group == 20:
        name = "Men's +20"
    elif group == 22:
        name = "Petite Women's +20"
    elif group == 24:
        name = "Petite Women's +40"
    else:
        name = ""
    return name
df["product_name"] = df["grouped_product"].apply(create_name)
df["product_name"].value_counts()

### Modify Zipcode

In [20]:
def modify_zipcode(zipcode):
    new_zip = 0
    if zipcode == "93807":
        new_zip = "92807"
    elif zipcode == "91491":
        new_zip = "91403"
    elif zipcode == "96251":
        new_zip = "96267"
    elif zipcode == "606041":
        new_zip = "60641"
    elif zipcode == "60483":
        new_zip = "60438"
    elif zipcode == "75036":
        new_zip = "75035"
    else:
        new_zip = zipcode
    return new_zip
df["shipping_postal_code"] = df["shipping_postal_code"].apply(modify_zipcode)
df["billing_postal_code"] = df["billing_postal_code"].apply(modify_zipcode)

("92807" in set(df["billing_postal_code"]),
"91403" in set(df["billing_postal_code"]),
"96267" in set(df["billing_postal_code"]),
"60641" in set(df["billing_postal_code"]),
"60438" in set(df["billing_postal_code"]),
"75035" in set(df["billing_postal_code"]))

In [ ]:
df.to_csv("Cleaned_V1.csv",index=False)

### Split zipcode

In [50]:
def split_zipcode(zipcode):
    new_zip = ""
    if type(zipcode) == str: #zipcode exists
        if zipcode.find("-")==5: # there's "-"
            new_zip = zipcode.split("-")[0]
        else: #there's no "-" in string zipcode
            new_zip = zipcode
    else: #empty zipcode
        new_zip = zipcode
    return new_zip

In [51]:
df.billing_postal_code = df.billing_postal_code.apply(split_zipcode)

In [55]:
df.billing_postal_code.replace({"331266070":"33126"},inplace = True)

In [19]:
df.billing_postal_code.value_counts()

In [62]:
df.shipping_postal_code = df.shipping_postal_code.apply(split_zipcode)

In [63]:
df.to_csv("Cleaned_V1.csv",index=False)

### Join dataset :)

In [7]:
df_1 = pd.read_csv("Cleaned_V1.csv")
df_2 = pd.read_csv("USZipCodes_subset.csv")
df_2.Zipcode = df_2.Zipcode.apply(lambda x: str(x))
result = pd.merge(df_1, df_2, left_on='billing_postal_code', right_on = "Zipcode", how='left')

In [18]:
result.tail()

In [10]:
result.to_csv("V1_Zip.csv",index=False)

### Group sku to product with size

In [13]:
df = pd.read_csv("V1_Zip.csv")

In [17]:
df.head(1)

In [16]:
df.columns

In [24]:
df_sku = pd.read_excel("AGOGIE SKU - UPC.xlsx")

In [15]:
df_sku.head(5)

In [26]:
df_sku.columns = df_sku.iloc[2]

In [14]:
df_sku.head(5)

In [28]:
df_sku = df_sku.drop([0,1,2])

In [13]:
df_sku.head(3)

In [48]:
df_sku_men = df_sku[['Item#_M','Description_M','UPC#_M']]
df_sku_men = df_sku_men.reset_index(drop=True)
df_sku_men = df_sku_men.drop([7,15,23,31,39,40])
df_sku_men = df_sku_men.reset_index(drop=True)

In [12]:
df_sku_men

In [59]:
df_sku_women = df_sku[['Item#_W','Description_W','UPC#_W']]
df_sku_women = df_sku_women.reset_index(drop=True)
df_sku_women = df_sku_women.drop([6,7,15,22,23])
df_sku_women = df_sku_women.reset_index(drop=True)
df_sku_women = df_sku_women.iloc[0:26]

In [11]:
df_sku_women

In [71]:
df_sku_petite = df_sku[['Item#_P','Description_P','UPC#_P']]
df_sku_petite = df_sku_petite.drop(10)
df_sku_petite = df_sku_petite.reset_index(drop=True)
df_sku_petite = df_sku_petite.iloc[0:14]

In [10]:
df_sku_petite

In [9]:
size_dic = {}
def sku2size(row): #size:'Women's Petite XS +20'
    size_dic[row['Item#_M']] = row['Description_M']
df_sku_men.apply(sku2size,axis=1) 
size_dic

In [8]:
def sku2size(row): #size:'Women's Petite XS +20'
    size_dic[str(row['Item#_W'])] = row['Description_W']
df_sku_women.apply(sku2size,axis=1) 
size_dic

In [7]:
def sku2size(row): #size:'Women's Petite XS +20'
    size_dic[row['Item#_P']] = row['Description_P']
df_sku_petite.apply(sku2size,axis=1) 
size_dic

In [6]:
size_dic['41700']

In [5]:
str(int(df.variant_sku[13752]))

In [105]:
def variant_sku2size(row): #size:'Women's Petite XS +20'
    size = size_dic[str(int(row['variant_sku']))] #variant_sku
    return size

In [111]:
df_sku_men.columns = ['Item#','Description','UPC#']
df_sku_women.columns = ['Item#','Description','UPC#']
df_sku_petite.columns = ['Item#','Description','UPC#']
frames = [df_sku_men,df_sku_women,df_sku_petite]
df_skus = pd.concat(frames)

In [4]:
df_skus

In [113]:
df_skus.to_csv("sku2sizeprod.csv",index=False)